In [1]:
!pip install pennylane

**Introduction to Applications of Quantum Computing to Quantum Chemistry**

**Problem 2: challenge**

**Item (c)**

In this notebook we implement a trotterization procedure, without the use of
qml.ApproxTimeEvolution, qml.evolve nor qml.QubitUnitary. The first cell loads the necessary libraries, the second implements the circuit for trotterization, and the last cell simply runs the circuit and prints the respective amplitude probabilities.


In [2]:
import pennylane as qml
from pennylane import numpy as pnp

Since the operators XX and ZZ satisfy A² = 1 (identity operator), we can make use of the closed form for the exponential of such operators:

$$
\exp^{i \theta M} = I \cos(\theta) + iM\sin(\theta).
$$

For the XX operator we identify:

$$
\theta = -\frac{\alpha t}{n},
$$

$$
M = X \otimes X.
$$

Then, we can do the same for the ZZ operator, such that in our trotterization we would have the product

$$
∏_{j=1}^{n} A_2(\beta,t) A_1(\alpha, t)
$$

However, it is easier to use the functions qml.IsingXX and qml.IsingZZ, which readily exponentiate the tensor products XX and ZZ with an angle of phi/2:

$$
XX(\phi) = \exp\left( -i \frac{\phi}{2} (X \otimes X) \right) =
\begin{bmatrix}
\cos(\phi/2) & 0 & 0 & -i \sin(\phi/2) \\
0 & \cos(\phi/2) & -i \sin(\phi/2) & 0 \\
0 & -i \sin(\phi/2) & \cos(\phi/2) & 0 \\
-i \sin(\phi/2) & 0 & 0 & \cos(\phi/2)
\end{bmatrix}.
$$

Note that we must implement the code with a double angle, and without the negative sign mentioned above.

In [3]:
dev = qml.device("default.qubit", wires=2)
@qml.qnode(dev)
def circuit(alpha, beta, t, n):
  #define new angles:
  theta_1 = 2*alpha*t/n
  theta_2 = 2*beta*t/n

  # Apply the trotterization
  for j in range(n):
      # Exponentiate the PauliX tensor product term: exp(-i*theta_1*X0*X1)
      qml.IsingXX(theta_1, wires=[0, 1])

      # Exponentiate the PauliZ tensor product term: exp(-i*theta_2*Z0*Z1)
      qml.IsingZZ(theta_2, wires=[0, 1])

  return qml.probs(wires=[0, 1])

# Since when computing amplitude probabilities we only need to measure in the Z-basis, this criterion of the exercise is readily fulfilled.


In [4]:
# Choose parameters and run the trotterization circuit

trotter_probs = circuit(0.1, 0.01, 1, 200)
basis_states = ['00', '01', '10', '11']

for i in range(len(trotter_probs)):
  print(f'Prob({basis_states[i]}) = {trotter_probs[i]:.4f}')

Prob(00) = 0.9900
Prob(01) = 0.0000
Prob(10) = 0.0000
Prob(11) = 0.0100
